# Multiple Linear Regression

In [2]:
import csv
import pandas as pd
import numpy as np
import sklearn
from sklearn import *

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/emily-air13/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [48]:
pd.set_option('display.max_columns', 65)

df = pd.read_csv('clean_data.csv', low_memory=False)

Variables in the Cleaned Data:

Location & Zoning Data (Potential Categorical X):
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>fips</li>
          <li>latitude</li>
        <li>longitude</li>
         <li>propertycountylandusecode</li>
         
         
         </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>propertylandusetypeid</li>
         <li>rawcensustractandblock</li>
         <li>regionidcity</li>
         <li>zipcode</li>
        </ul>
    </div>
</div>

Categorical X:

<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
         <li>airconditioningtypeid</li>
         <li>hashottuborspa</li>
         <li>heatingorsystemtypeid</li>
        <li>fireplaceflag</li>
        <li>assessmentyear</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>taxdelinquencyflag</li>
        <li>setyear</li>
        <li>pooltype</li> 
        <li>transactiondate</li>
        </ul>
    </div>
</div>


Numerical X:
<div style="-webkit-column-count: 2; -moz-column-count: 2; column-count: 2; -webkit-column-rule: 1px dotted #e0e0e0; -moz-column-rule: 1px dotted #e0e0e0; column-rule: 1px dotted #e0e0e0;">
    <div style="display: inline-block;">
        <ul>
        <li>bedroomcnt</li>
        <li>buildingqualitytypeid</li>
        <li>calculatedbathnbr</li>
        <li>calculatedfinishedsquarefeet</li>
        <li>fireplacecnt</li>
        <li>fullbathcnt</li>
        <li>garagecarcnt</li>
        <li>garagetotalsqft</li>
        <li>lotsizesquarefeet</li>
        <li>poolcnt</li>
        </ul>
    </div>
    <div style="display: inline-block;">
        <ul>
        <li>roomcnt</li>
        <li>unitcnt</li>
        <li>yearbuilt</li>
        <li>numberofstories</li>
        <li>structuretaxvaluedollarcnt</li>
        <li>taxvaluedollarcnt</li>
        <li>landtaxvaluedollarcnt</li>
        <li>taxamount</li>
        <li>taxdelinquencyyear TO REMOVE</li> 
        </ul>
    </div>
</div>
           
                        
Y:
* logerror

In [179]:
pool = pd.get_dummies(df['pooltype'])
df = df.join(pool)


In [49]:
def hottub(a):
    if a:
        return 1
    else:
        return 0
    
df['hashottuborspa'] = df['hashottuborspa'].map(lambda a: hottub(a))

def fire(a):
    if a:
        return 1
    else:
        return 0

df['fireplaceflag'] = df['fireplaceflag'].map(lambda a: fire(a))

def tax(a):
    if a=='Y':
        return 1
    else:
        return 0

df['taxdelinquencyflag'] = df['taxdelinquencyflag'].map(lambda a: tax(a))

fips = pd.get_dummies(df['fips'])
df = df.join(fips)

yr = pd.get_dummies(df['setyear'])
df = df.join(yr)

ac = pd.get_dummies(df['airconditioningtypeid'], prefix='ac')
df = df.join(ac)

heat = pd.get_dummies(df['heatingorsystemtypeid'], prefix='heat')
df=df.join(heat)

df.head()


,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,transactiondate,setyear,pooltype,zipcode,6037.0,6059.0,6111.0,2016,2017,ac_1.0,ac_3.0,ac_5.0,ac_9.0,ac_11.0,ac_13.0,heat_1.0,heat_2.0,heat_6.0,heat_7.0,heat_10.0,heat_11.0,heat_12.0,heat_13.0,heat_14.0,heat_18.0,heat_20.0,heat_24.0
0,17073783,5.0,3.0,6.015461,2.5,1264.0,6111.0,0.0,2.0,2.0,434.0,0,2.0,34.303597,-119.287236,1735.0,0.0,1128,265.0,61110022003007,34543.0,5.0,1.0,1986.0,2.0,0,115087.0,191811.0,2015.0,76724.0,2015.06,0,NaN,0.0953,2016-01-27,2016,0,93001,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,17088994,5.0,2.0,6.015461,1.0,777.0,6111.0,0.0,1.0,1.0,434.0,0,2.0,34.272866,-119.198911,7200.0,0.0,1129,266.0,61110015031002,34543.0,4.0,1.0,1990.0,1.0,0,143809.0,239679.0,2015.0,95870.0,2581.30,0,NaN,0.0198,2016-03-30,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,17100444,5.0,3.0,6.015461,2.0,1101.0,6111.0,0.0,2.0,2.0,441.0,0,2.0,34.340801,-119.079610,6569.0,0.0,1111,261.0,61110007011007,26965.0,5.0,1.0,1956.0,1.0,0,33619.0,47853.0,2015.0,14234.0,591.64,0,NaN,0.0060,2016-05-27,2016,0,93066,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,17102429,5.0,2.0,6.015461,1.5,1554.0,6111.0,1.0,1.0,2.0,460.0,0,2.0,34.354313,-119.076405,7400.0,0.0,1110,261.0,61110008002013,26965.0,5.0,1.0,1965.0,1.0,1,45609.0,62914.0,2015.0,17305.0,682.78,0,NaN,-0.0566,2016-06-07,2016,0,93060,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,17109604,5.0,4.0,6.015461,2.5,2415.0,6111.0,1.0,2.0,2.0,665.0,0,2.0,34.266578,-119.165392,6326.0,0.0,1111,261.0,61110014021007,34543.0,8.0,1.0,1984.0,2.0,1,277000.0,554000.0,2015.0,277000.0,5886.92,0,NaN,0.0573,2016-08-08,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [180]:
df.head()

,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertycountylandusecode,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,transactiondate,setyear,pooltype,zipcode,6037.0,6059.0,6111.0,2016,2017,ac_1.0,ac_3.0,ac_5.0,ac_9.0,ac_11.0,ac_13.0,heat_1.0,heat_2.0,heat_6.0,heat_7.0,heat_10.0,heat_11.0,heat_12.0,heat_13.0,heat_14.0,heat_18.0,heat_20.0,heat_24.0,0,2,7,10
0,17073783,5.0,3.0,6.015461,2.5,1264.0,6111.0,0.0,2.0,2.0,434.0,0,2.0,34.303597,-119.287236,1735.0,0.0,1128,265.0,61110022003007,34543.0,5.0,1.0,1986.0,2.0,0,115087.0,191811.0,2015.0,76724.0,2015.06,0,NaN,0.0953,2016-01-27,2016,0,93001,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,17088994,5.0,2.0,6.015461,1.0,777.0,6111.0,0.0,1.0,1.0,434.0,0,2.0,34.272866,-119.198911,7200.0,0.0,1129,266.0,61110015031002,34543.0,4.0,1.0,1990.0,1.0,0,143809.0,239679.0,2015.0,95870.0,2581.30,0,NaN,0.0198,2016-03-30,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,17100444,5.0,3.0,6.015461,2.0,1101.0,6111.0,0.0,2.0,2.0,441.0,0,2.0,34.340801,-119.079610,6569.0,0.0,1111,261.0,61110007011007,26965.0,5.0,1.0,1956.0,1.0,0,33619.0,47853.0,2015.0,14234.0,591.64,0,NaN,0.0060,2016-05-27,2016,0,93066,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,17102429,5.0,2.0,6.015461,1.5,1554.0,6111.0,1.0,1.0,2.0,460.0,0,2.0,34.354313,-119.076405,7400.0,0.0,1110,261.0,61110008002013,26965.0,5.0,1.0,1965.0,1.0,1,45609.0,62914.0,2015.0,17305.0,682.78,0,NaN,-0.0566,2016-06-07,2016,0,93060,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,17109604,5.0,4.0,6.015461,2.5,2415.0,6111.0,1.0,2.0,2.0,665.0,0,2.0,34.266578,-119.165392,6326.0,0.0,1111,261.0,61110014021007,34543.0,8.0,1.0,1984.0,2.0,1,277000.0,554000.0,2015.0,277000.0,5886.92,0,NaN,0.0573,2016-08-08,2016,0,93004,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [50]:
msk = np.random.rand(len(df)) < 0.8

In [51]:
train = df[msk]
test = df[~msk]

In [52]:
train.describe()

,parcelid,airconditioningtypeid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,propertylandusetypeid,rawcensustractandblock,regionidcity,roomcnt,unitcnt,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,logerror,setyear,pooltype,zipcode,6037.0,6059.0,6111.0,2016,2017,ac_1.0,ac_3.0,ac_5.0,ac_9.0,ac_11.0,ac_13.0,heat_1.0,heat_2.0,heat_6.0,heat_7.0,heat_10.0,heat_11.0,heat_12.0,heat_13.0,heat_14.0,heat_18.0,heat_20.0,heat_24.0
count,1.344180e+05,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.00000,134418.000000,134418.000000,1.344180e+05,134418.000000,134418.000000,1.344180e+05,131819.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,1.344180e+05,1.344180e+05,134418.000000,1.344180e+05,134418.000000,134418.000000,3764.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.0,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000,134418.0,134418.000000,134418.000000,134418.000000,134418.000000,134418.000000
mean,1.300138e+07,3.978857,3.042792,6.014039,2.309241,1777.997560,6048.846985,0.127609,2.242601,0.600217,157.452863,0.023501,3.21297,34.006564,-118.200831,2.605276e+04,0.203916,261.820017,6.049156e+13,33736.411875,5.581864,1.069336,1968.555811,1.099756,0.106764,1.843730e+05,4.728271e+05,2015.462669,2.888325e+05,5989.551282,0.028002,2013.784803,0.013789,2016.462669,1.455534,91596.937746,0.650672,0.269294,0.080034,0.537331,0.462669,0.296932,0.0,0.681985,0.000156,0.000729,0.020198,0.000141,0.799335,0.010549,0.176033,0.000030,0.000015,0.0,0.000803,0.000015,0.000298,0.001146,0.011635
std,3.145526e+06,2.242687,1.153189,1.504190,0.972988,940.634141,20.694355,0.402941,0.959543,0.921874,238.612536,0.151490,3.04629,0.265228,0.359807,8.540384e+04,0.402909,5.263489,2.053674e+11,47120.101072,1.974439,0.371586,23.686272,0.318498,0.308814,2.208302e+05,6.106546e+05,0.498606,4.537128e+05,7290.353818,0.164980,1.685764,0.166799,0.498606,2.867673,1083.380871,0.476760,0.443595,0.271347,0.498606,0.498606,0.456908,0.0,0.465707,0.012498,0.026992,0.140678,0.011888,0.400499,0.102166,0.380850,0.005455,0.003857,0.0,0.028334,0.003857,0.017248,0.033829,0.107238
min,1.071174e+07,1.000000,0.000000,1.000000,1.000000,120.000000,6037.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.00000,33.339295,-119.475416,2.360000e+02,0.000000,31.000000,6.037101e+13,3491.000000,1.000000,1.000000,1824.000000,1.000000,0.000000,4.400000e+01,2.200000e+01,2015.000000,2.200000e+01,19.920000,0.000000,1999.000000,-4.605000,2016.000000,0.000000,90001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.154914e+07,1.000000,2.000000,6.000000,2.000000,1184.000000,6037.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.00000,33.812462,-118.413314,5.962000e+03,0.000000,261.000000,6.037312e+13,12447.000000,4.000000,1.000000,1953.000000,1.000000,0.000000,8.274000e+04,2.021570e+05,2015.000000,8.323950e+04,2792.337500,0.000000,2013.000000,-0.025300,2016.000000,0.000000,90660.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.254332e+07,5.000000,3.000000,6.015461,2.000000,1541.000000,6037.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.00000,34.021534,-118.176850,

In [68]:
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31,38,39,40,41,42,43,44,45,46,47,48,29,49,50,51,52,53,54,55,56,57,58,59,60]], array_train[:,33])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [69]:
lm.intercept_

0.85584965689776482

In [70]:
lm.coef_

array([  6.00854964e-04,  -1.51104683e-03,  -2.78895200e-03,
         1.36852024e-05,   1.40746656e-03,   1.57388848e-03,
         3.84818230e-03,  -2.41411447e-05,  -3.14136833e-03,
         6.28309105e-03,   3.70718167e-08,  -8.88502794e-03,
         1.19632253e-05,  -8.02170852e-03,   1.70814354e-04,
        -3.66995684e-07,   3.94859454e-07,  -1.81146375e-07,
        -3.27858077e-06,  -1.27641564e-02,  -9.06356138e-03,
         1.69701785e-02,  -4.78419360e-03,  -2.21093216e-03,
         6.99512576e-03,  -2.93096762e-03,   2.93096762e-03,
         8.04751933e-04,   1.73472348e-18,   2.09347438e-03,
        -1.90015498e-02,   1.59558509e-02,   1.47472618e-04,
        -1.81067480e-07,   5.40706732e-02,   1.18972874e-02,
         1.27060739e-02,   1.08268334e-02,  -4.22572953e-02,
        -5.24371882e-02,   0.00000000e+00,  -1.34923978e-03,
         3.51314765e-04,   1.76401301e-02,  -2.58452385e-03,
        -8.86406576e-03])

In [71]:
array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31,38,39,40,41,42,43,44,45,46,47,48,29,49,50,51,52,53,54,55,56,57,58,59,60]])

In [57]:
from sklearn.metrics import mean_squared_error
from math import sqrt

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))
 

0.15994609372234428


In [ ]:

from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFECV(estimator, step=1, cv=5)
selector = selector.fit(train.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31]], train.iloc[:,33])


In [ ]:
selector.support_ 
selector.ranking_


In [64]:
from sklearn import linear_model
clf = linear_model.Lasso(tol=1) #alpha=0.1
clf.fit(train.iloc[:,[2,3,4,5,7,8,9,10,13,14,15,16,21,22,24,26,27,29,30,11,25,31,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]], train.iloc[:,33])


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=1, warm_start=False)

In [65]:
clf.intercept_

0.0018323104777448144

In [66]:
clf.coef_ #5, 15 26,27,29,30

array([  0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         8.23493681e-06,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   2.53354944e-08,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
        -3.87351728e-09,  -1.69229080e-09,  -1.08049856e-09,
        -2.53536284e-07,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00])

In [67]:
from sklearn.metrics import mean_squared_error
from math import sqrt


array_test = test.values
array_train = train.values

reg=linear_model.LinearRegression()
reg.fit(array_train[:,[5,15,26,27,29,30]], array_train[:,33])
p = reg.predict(array_test[:,[5,15,26,27,29,30]])
print(sqrt(mean_squared_error(array_test[:,33],p)))

0.16005016011359782


In [ ]:
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)

In [140]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[3,7,11,13,15,26,29,38,39,41,42,43,45,48,50,51,52,53,54,55,56,57,58,59,60]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[3,7,11,13,15,26,29,38,39,41,42,43,45,48,50,51,52,53,54,55,56,57,58,59,60]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))


0.16029404835498545


All:               0.15994609372231042
Remove 2:          0.1599461468254335
Remove 3:          0.15994139947869387
Add 3, remove 4:   0.15994601407578807
Remove 5:          0.16002736613201468
Remove 7:          0.16002560995327034
Add 7, remove 8:   0.16004570567106657
Remove 9:          0.1600467659719313
Remove 10:         0.16005384969415262 
Remove 11:         0.1600511367437618
Add 11, remove 13: 0.16005327618207021
Add 13, remove 14: 0.16005644584053777 
Remove 15:         0.16004462485611465
Add 15, remove 16: 0.16007533377960984 
Remove 21:         0.16017049006374046 
Remove 22:         0.16017170517249144
Remove 24:         0.1601783899872867
Remove 25:         0.1601796287662811
Remove 26:         0.16017706982032512
Add 26, remove 27: 0.16018026912270933  
Remove 29:         0.16017920035754296
Add 29, remove 30: 0.16023854885917077       
Remove 31:         0.1602925417914655
Remove 38:         0.16029254179143937
Add 38, remove 39: 0.16029254179142366
Add 39, remove 40: 0.1602925417914472
Remove 41:         0.16029254179144642
Add 41, remove 42: 0.16029254179144642
Add 42, remove 43: 0.1602925385331169
Add 43, remove 44: 0.16029254179144858
Remove 45:         0.16029251923916113
Add 45, remove 46: 0.1602927737180532
Remove 47:         0.16029369722853465
Remove 48:         0.16029366154133506
Add 48, remove 49: 0.16029404835498545   ****
Remove 50:         0.16029224207830797
Add 50, remove 51: 0.1602921893748259
Add 51, remove 52: 0.16029202797063402
Add 52, remove 53: 0.1602927328930735
Add 53, remove 54: 0.16029339478868923
Add 54, remove 55: 0.1602940483549817
Add 55, remove 56: 0.16029255811077978
Add 56, remove 57: 0.16029366638148124
Add 57, remove 58: 0.16029246489881105
Add 58, remove 59: 0.16029228381359256
Add 59, remove 60: 0.16029145563460911


Best: 0.16029404835498545

buildingqualitytypeid
fireplacecnt
hashottuborspa
latitude
lotsizesqarefeet
structuretaxvaluedollarcnt
landtaxvaluedollarcnt
FIPS codes 6037 and 6059
Set Year
AC types None, Yes, Central (5, 13, 1)
Heat except baseboard heat (1)

In [141]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16030219724685932


In [142]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43,11]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43,11]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16030273953373883


In [144]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43,11,3]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43,11,3]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.160304916311392


In [146]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43,11,3,15]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43,11,3,15]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16031790171710306


In [147]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43,11,3,15,26]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43,11,3,15,26]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16032524933869377


In [150]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[42,43,11,3,15,26,49,50,51,52,53,54,55,56,57,58,59,60]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[42,43,11,3,15,26,49,50,51,52,53,54,55,56,57,58,59,60]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16033215899151984


In [177]:
lm.coef_

array([ -1.53422972e-03,  -7.47628117e-03,  -3.86553879e-03,
         2.15165278e-08,   1.39227432e-08,   7.27889901e-03,
        -1.78757213e-03,   5.13507524e-02,   1.21968638e-02,
         9.61003664e-03,   8.39647764e-03,  -4.32864975e-02,
        -4.39234844e-02,   9.54097912e-18,   1.45245053e-03,
        -4.55601526e-03,   2.00155766e-02,  -2.27173589e-03,
        -8.98442452e-03])

In [178]:
lm.intercept_

0.13744687268075786

In [153]:
# Manual Selection
lm=linear_model.LinearRegression()
array_train=train.values
lm.fit(array_train[:,[3,11,13,15,26,42,43,49,50,51,52,53,54,55,56,57,58,59,60]], array_train[:,33])


#array_test = test.values

# Make predictions using the testing set
log_pred = lm.predict(test.iloc[:,[3,11,13,15,26,42,43,49,50,51,52,53,54,55,56,57,58,59,60]])

print(sqrt(mean_squared_error(array_test[:,33], log_pred)))

0.16033999669758048


In [176]:
# use cannot use MAPE because cannot divide by zero, use SMAPE instead
# https://math.stackexchange.com/questions/677852/how-to-calculate-relative-error-when-true-value-is-zero
# https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error

y_true = np.array(array_test[:,33])
y_score = np.array(log_pred)

np.mean(np.abs(y_true - y_score) / (np.abs(y_true) + np.abs(y_score))) * 100

72.32086013904555

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200)
forest.fit(x[:, None], y)

xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = model(xfit, sigma=0)

plt.errorbar(x, y, 0.3, fmt='o', alpha=0.5)
plt.plot(xfit, yfit, '-r');
plt.plot(xfit, ytrue, '-k', alpha=0.5);

